In [1]:
import dask
from dask.diagnostics import ProgressBar
from datatree import DataTree
import itertools as it
from pathlib import Path
import geopandas as gpd
from clisops.core.subset import create_weight_masks, subset_bbox
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import xclim as xc
import xclim.ensembles as xce
import xesmf as xe
import xscen as xs
dask.config.set(num_workers=20)
ProgressBar().register()

2023-03-29 12:33:29,590 - pint.util - WARNING - Redefining 'year' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,595 - pint.util - WARNING - Redefining 'yr' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,599 - pint.util - WARNING - Redefining 'C' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,604 - pint.util - WARNING - Redefining 'd' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,608 - pint.util - WARNING - Redefining 'h' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,612 - pint.util - WARNING - Redefining 'degrees_north' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,615 - pint.util - WARNING - Redefining 'degrees_east' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-03-29 12:33:29,619 - pint.util - WARNING - Redefining '[speed]' (<class 'pint.delegates.txt_d

In [10]:
probreg = dict(
    name="prob-reg",
    method="bbox",
    bbox=dict(
        lat_bnds=[45, 47],
        lon_bnds=[-75, -70]
    ),
)
regions = gpd.read_file('/exec/pbourg/PC/20230226/regions_simplified_admin.geojson')
estrie = regions.iloc[[29]]
estrie
# xs.load_config('/home/pbourg/Projets/ESPO-G/configuration/config_ESPO-G_RDRS.yml', '/home/pbourg/Projets/ESPO-G/configuration/paths_ESPO-G.yml')

,id,name,geometry
29,30,Estrie,"POLYGON ((-71.06889 45.92744, -71.05771 45.938..."


In [3]:
cat = xs.DataCatalog('/jarre/scenario/jlavoie/ESPO-G6/cat_ESPO-G6_RDRS.json')

In [4]:
scats = xs.search_data_catalogs(
    cat,
    variables_and_freqs={'tasmax': 'D', 'tasmin': 'D'},
    other_search_criteria=dict(processing_level='regridded', domain='prob-reg', experiment='ssp245'),
)
ref_path = '/jarre/scenario/jlavoie/ESPO-G6/reference/ref_prob-reg_{cal}.zarr/'

2023-03-29 12:34:06,507 - xscen.extract - INFO - Catalog opened: <cat_ESPO-G6_extractamno catalog with 699 dataset(s) from 715 asset(s)> from 1 files.
2023-03-29 12:34:06,531 - xscen.extract - INFO - 13 assets matched the criteria : {'processing_level': 'regridded', 'domain': 'prob-reg', 'experiment': 'ssp245'}.
2023-03-29 12:34:06,543 - xscen.extract - INFO - Iterating over 13 potential datasets.
2023-03-29 12:34:08,184 - xscen.extract - INFO - Found 13 with all variables requested and corresponding to the criteria.


In [9]:
for dsid, scat in scats.items():
    ds = xs.extract_dataset(scat)['D']
    
    for var in ['tasmax', 'tasmin']:
        simcal = xc.core.calendar.get_calendar(ds)
        refcal = xs.utils.minimum_calendar(simcal, 'noleap')
        ds_ref = xr.open_zarr(ref_path.format(cal=refcal))


        ds_tr = xs.train(
            dref=ds_ref,
            dhist=ds,
            var=[var],
            reference_period=["1991", "2020"],
            method="DetrendedQuantileMapping",
            group={"group": "time.dayofyear", "window": 31},
            xclim_train_args={"kind": "+", "nquantiles": 50}
        )

        # adjust
        ds_scen = xs.adjust(
            dsim=ds,
            dtrain=ds_tr,
            simulation_period=[["1950", "2100"]],
            xclim_adjust_args={
                "detrend": {
                    "LoessDetrend": {
                      "f": 0.2,
                      "niter": 1,
                      "d": 0,
                      "weights": "tricube"
                    }
                },
                "interp": "nearest",
                "extrapolation": "constant",
            },
            bias_adjust_institution="Ouranos",
            bias_adjust_project="ESPO-G6-TN"
        )

        ds_scen = xs.clean_up(
            ds=ds_scen.drop_vars(['loc', 'lon', 'lat']),
            maybe_unstack_dict={
                "stack_drop_nans": True,
                "rechunk": {
                  "rlat": 20,
                  "rlon": 20,
                  "time": -1
                }
            },
            to_level="cleaned_up",
            add_attrs={
                var: {
                    "standard_name": "air_temperature"
                }
            }
        )
        ds_scen.to_zarr(f'/exec/pbourg/ESPO-G/NAM_SPLIT/prob-reg/day_{dsid}_{var}.zarr')

[########################################] | 100% Completed | 108.35 ms
[########################################] | 100% Completed | 102.68 ms
[########################################] | 100% Completed | 103.00 ms
[########################################] | 100% Completed | 102.84 ms
[########################################] | 100% Completed | 102.60 ms
[########################################] | 100% Completed | 105.62 ms
[########################################] | 100% Completed | 102.68 ms
[########################################] | 100% Completed | 102.38 ms
[########################################] | 100% Completed | 103.12 ms
2023-03-29 12:36:47,607 - xscen.utils - INFO - Dataset unstacked using no coords argument.


/exec/pbourg/.conda/xscen/lib/python3.10/site-packages/xarray/core/dataset.py:4895: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')


[########################################] | 100% Completed | 328.32 s
[########################################] | 100% Completed | 105.81 ms
[########################################] | 100% Completed | 102.88 ms
[########################################] | 100% Completed | 102.56 ms
[########################################] | 100% Completed | 103.93 ms
[########################################] | 100% Completed | 102.78 ms
[########################################] | 100% Completed | 102.45 ms
[########################################] | 100% Completed | 102.80 ms
[########################################] | 100% Completed | 101.88 ms
2023-03-29 12:42:24,472 - xscen.utils - INFO - Dataset unstacked using no coords argument.
[########################################] | 100% Completed | 330.66 s
[########################################] | 100% Completed | 569.56 ms
[########################################] | 100% Completed | 102.54 ms
[########################################] | 10

[########################################] | 100% Completed | 102.67 ms
[########################################] | 100% Completed | 101.53 ms
[########################################] | 100% Completed | 102.17 ms
2023-03-29 13:37:52,566 - xscen.utils - INFO - Dataset unstacked using no coords argument.
[########################################] | 100% Completed | 312.00 s
[########################################] | 100% Completed | 303.53 ms
[########################################] | 100% Completed | 102.90 ms
[########################################] | 100% Completed | 102.50 ms
[########################################] | 100% Completed | 102.16 ms
[########################################] | 100% Completed | 102.54 ms
[########################################] | 100% Completed | 102.23 ms
[########################################] | 100% Completed | 102.89 ms
[########################################] | 100% Completed | 103.20 ms
[########################################] | 1

[########################################] | 100% Completed | 102.49 ms
[########################################] | 100% Completed | 102.38 ms
[########################################] | 100% Completed | 101.96 ms
[########################################] | 100% Completed | 102.97 ms
[########################################] | 100% Completed | 102.03 ms
[########################################] | 100% Completed | 103.48 ms
[########################################] | 100% Completed | 102.09 ms
[########################################] | 100% Completed | 102.38 ms
2023-03-29 14:40:33,897 - xscen.utils - INFO - Dataset unstacked using no coords argument.
[########################################] | 100% Completed | 324.61 s
[########################################] | 100% Completed | 16.29 s
[########################################] | 100% Completed | 17.12 ss
[########################################] | 100% Completed | 12.67 s
[########################################] | 100% C

In [15]:
outroot = Path('/exec/pbourg/ESPO-G/indsanal/')
ds = xr.open_zarr('/jarre/scenario/jlavoie/ESPO-G6/NAM_SPLIT/prob-reg/day_CMIP6_ScenarioMIP_BCC_BCC-CSM2-MR_ssp245_r1i1p1f1_global_1950-2100.zarr/')
ds_bnds = xs.regrid.create_bounds_rotated_pole(ds)

In [16]:
ds_bnds

<xarray.Dataset>
Dimensions:       (rlon: 43, rlat: 33, bounds: 4)
Coordinates:
  * rlat          (rlat) float64 -11.61 -11.52 -11.43 ... -8.91 -8.82 -8.73
  * rlon          (rlon) float64 372.0 372.1 372.2 372.3 ... 375.6 375.7 375.8
    lat           (rlat, rlon) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    rotated_pole  float32 9.969e+36
    lon           (rlat, rlon) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
Dimensions without coordinates: bounds
Data variables:
    lon_bounds    (rlat, rlon, bounds) float64 -75.66 -75.62 ... -69.16 -69.2
    lat_bounds    (rlat, rlon, bounds) float64 45.09 45.18 45.16 ... 46.81 46.72

In [19]:
outroot = Path('/exec/pbourg/ESPO-G/indsanal/')
ds = xr.open_zarr('/jarre/scenario/jlavoie/ESPO-G6/NAM_SPLIT/prob-reg/day_CMIP6_ScenarioMIP_BCC_BCC-CSM2-MR_ssp245_r1i1p1f1_global_1950-2100.zarr/')
ds_bnds = xs.regrid.create_bounds_rotated_pole(ds)
ds_bnds.lon.load(), ds_bnds.lat.load()
for p in Path('/exec/pbourg/ESPO-G/NAM_SPLIT/prob-reg/').glob('day_*_tasmax.zarr'):
    tx = xr.open_zarr(p)
    tn = xr.open_zarr(p.parent / p.name.replace('tasmax', 'tasmin'))
    day = xr.merge(
        [
            tx.tasmax,
            tn.tasmin,
            (tx.tasmax - tn.tasmin).rename('dtr').assign_attrs(units='K')
        ]
    ).update(ds_bnds)
    mon = xr.merge(
        [
            xc.atmos.daily_freezethaw_cycles(ds=day, freq='MS'),
            xc.atmos.daily_temperature_range(ds=day, freq='MS'),
            create_weight_masks(day, estrie).squeeze('geom', drop=True).rename('mask')
        ]
    )
    dask.compute(
        day.to_zarr(
            outroot / p.name.replace('day_', 'day_ESPO-TN_').replace('tasmax', 'prob-reg'),
            compute=False
        ),
        mon.to_zarr(
            outroot / p.name.replace('day_', 'yr_ESPO-TN_').replace('tasmax', 'prob-reg'),
            compute=False
        )
    )

[########################################] | 100% Completed | 103.59 ms
[########################################] | 100% Completed | 167.20 s
[########################################] | 100% Completed | 167.77 s
[########################################] | 100% Completed | 167.95 s
[########################################] | 100% Completed | 160.19 s
[########################################] | 100% Completed | 165.85 s
[########################################] | 100% Completed | 169.84 s
[########################################] | 100% Completed | 164.10 s
[########################################] | 100% Completed | 166.26 s
[########################################] | 100% Completed | 168.27 s
[########################################] | 100% Completed | 167.31 s
[########################################] | 100% Completed | 154.55 s
[########################################] | 100% Completed | 157.99 s
[########################################] | 100% Completed | 160.02 s
